# HW1 Binary Search using Inverted Index

In [1]:
# Imports required libraries to execute the notebook
import time
import numpy as np
import pandas as pd
from gensim import corpora

In [2]:
# Imports the dictionary and document corpus from a preprocessed file
dictionary = corpora.Dictionary.load('resources/vocab.dict')
doc_corpus = corpora.MmCorpus('resources/doc_corpus.mm')

In [3]:
""" Generates the inverted index from the document corpus.

Args:
    doc_corpus (gensim.corpora.mmcorpus.MmCorpus): document corpus used to generate the inverted index.
    
Returns:
    inverted_matrix (numpy.ndarray): inverted index matrix with terms as the row index, 
        first column with the relative frequency and posting lists. The matrix is saved
        on disk when completely generated.
"""

# Creates a dictionary. Dictionary will hold token as key and list as value.
index_dictionary = {}

# Iterates over every document in the corpus
for doc_indx in range(len(doc_corpus)):    
    
    # Retrieves document from corpus using index
    document = doc_corpus[doc_indx]
    
    # Calculates document ID using the document index
    doc_id = doc_indx + 1
    
    # Iterates over every token present in document
    for token_indx in range(len(document)):
        
        # Converts token ID to token
        token = document[token_indx][0]
        
        if token not in index_dictionary.keys():
            index_dictionary[token] = [doc_id]
        else:
            index_dictionary[token].append(doc_id)
            
# Creates the numpy array
inverted_index = np.zeros((len(dictionary), len(doc_corpus) + 1), dtype=np.int_)
            
# Calculates the relative frequency and converts to numpy array
for key in index_dictionary.keys():
    
    relative_frequency = len(index_dictionary[key])
    
    inverted_index[key, 0] = relative_frequency
    
    inverted_index[key, 1:1+relative_frequency] = index_dictionary[key]

# Saves the inverted index matrix
np.save('./data/IImatrix.npy', inverted_index)

In [4]:
# Reads the inverted index from disk
inverted_index = np.load('./data/IImatrix.npy')

In [5]:
def merge(posting_list_0, posting_list_1, operator, n_docs, mod_term_0=False, mod_term_1=False):
    """ Merges two posting lists supporting a binary operator and modifiers.
    
    Args:
        posting_list_0 (list): list with the postings for the first term.
        posting_list_1 (list): list with the postings for the second term.
        operator (str): binary operator used to merge the lists. It could be
            either 'AND' or 'NOT'.
        n_docs (int): total number of documents in the corpus.
        mod_term_0 (bool): boolean modifier for the first posting list. If set to
            False, no modifying action is done. Otherwise, a NOT operator is used 
            for the posting lists.
        mod_term_0 (bool): boolean modifier for the second posting list. If set to
            False, no modifying action is done. Otherwise, a NOT operator is used 
            for the posting lists.
    
    Returns:
        list: merged posting lists.
    
    Raises:
        Exception: if the boolean operator is not valid.
    
    """
    
    # Checks operator
    if operator not in ["OR", "AND"]:
        raise Exception("Not valid boolean operator. Operator must be either 'OR' or 'AND'.")
        
    # Changes first posting lists if modifier is enabled
    if mod_term_0:
        posting_list_0 = [val for val in range(1, n_docs) if val not in posting_list_0]
    
    # Changes second posting lists if modifier is enabled
    if mod_term_1:
        posting_list_1 = [val for val in range(1, n_docs) if val not in posting_list_1]
          
    # Sets pl0 as the shortest and pl1 as the longest
    if len(posting_list_1) < len(posting_list_0):
        pl0, pl1 = posting_list_1, posting_list_0
    else:
        pl0, pl1 = posting_list_0, posting_list_1
        
    # Calculates the length of both lists
    len_pl0 = len(pl0)
    len_pl1 = len(pl1)
    
    # Creates the output list for merged posting lists  
    merged = []
        
    # Creates index list
    indexes = [0, 0]
    
    # Creates a stop condition for a disjunctive merge with an empty list
    if operator == "AND" and (len(pl0) == 0 or len(pl1) == 0):
        return []
    else:
        finish = False
    
    # Main merging loop
    while not finish:
        
        # Retrieves postings from lists
        posting0 = pl0[indexes[0]]
        posting1 = pl1[indexes[1]]
    
        # If merging using an AND operator
        if operator == "AND":
            
            # Adds posting to merged list
            if posting0 == posting1:
                merged.append(posting0)
            
            # Increments corresponding pointer
            if posting0 < posting1:
                indexes[0] += 1
            else:
                indexes[1] += 1
                
            # Verifies stop condition
            if indexes[0] == len_pl0 - 1 or indexes[1] == len_pl1 - 1:
                finish = True
        
        # If merging using an OR operator
        else:
            
            # Moves along first list
            if indexes[0] != len_pl0 - 1:
                
                # Checks condition for smaller list
                if posting0 < posting1:
                    merged.append(posting0)
                    indexes[0] += 1
                    
                # Checks condition for both lists
                elif posting0 == posting1:
                    merged.append(posting0)
                    indexes[0] += 1
                    indexes[1] += 1
                
                # Checks condition for longer list
                else:
                    merged.append(posting1)
                    indexes[1] += 1
                
            # Moves along second list
            else:
                merged.append(posting1)
                indexes[1] += 1
            
            # Checks stop condition
            if indexes[1] == len_pl1:
                finish = True

    # Returns output list for merge operation
    return merged

In [6]:
def inverted_index_query(inverted_index, query, n_docs, conjunctive=True):
    """ Executes a query for the inverted index.
    
    Args:
        inverted_index (numpy.ndarray): matrix with the inverted index. Assumed to have the row index as
            the term, the first column as the relative frequency and the remainder as the posting lists.
        query (list): list with the terms included in the query.
        n_docs (int): total number of documents in corpus.
        conjunctive (bool): performs the query in a conjunctive way if True, disjunct otherwise.
        
    Returns:
        list: with the query relevant documents.
    
    """
    
    # Array to store terms with relative frequency
    terms = []
    
    # Sets the boolean operator
    if conjunctive:
        operator = "AND"
    else:
        operator = "OR"
    
    # Retrieves relative frequency from 
    for term in query:
        rf = inverted_index[term][0]
        terms.append([term, rf])
    
    # Converts to numpy array
    terms = np.asarray(terms)
    
    # Orders the terms
    terms = terms[terms[:, 1].argsort()]
    
    # Gets the first posting list
    result = inverted_index[terms[0,0]][1::]
    
    # If there is more than one term, results are calculated
    if terms.shape[0] != 1:
        
        # Removes zeroes from first posting list
        result = result[result != 0]
        
        # Iterates over remaining posting lists
        for indx in range(1, terms.shape[0]):
            
            # Gets second posting list
            second = inverted_index[terms[indx, 0]][1::]
            
            # Removes zeroes from second posting list
            second = second[second != 0]

            # Merges the posting lists
            result = merge(result, second, operator, n_docs)
            
        # Returns the result for the lists
        return result
    
    # If only one term is present in query, its posting list is returned
    else:
        return result[result != 0]
    

In [7]:
# Reads query corpus
query_corpus = [[term[0] for term in query] for query in corpora.MmCorpus("./resources/query_corpus.mm")]

# Sets the query names from golden file
query_names = list(pd.read_csv("./data/relevance-judgments.tsv", sep='\t', names=['query', 'd']).loc[:, "query"])

In [8]:
import time

In [20]:
""" Executes the conjunctive queries and writes the results into a file.

Args:
    query_corpus (list): has lists whose elements are the terms used in the query.
    query_names (list): contains the names of the queries.
    
"""

# Sets a variable with the number of iterations to average
N = 10

# Initializes time value
cumulative_time = 0

# Runs the queries for a given time
for iteration in range(N):
    
    # Sets initial time
    initial_time = time.time()
    
    # Opens the results file
    conj_file = open("./results/BSII-AND-queries-results.tsv", "w")

    # Iterates over queries
    for query_indx in range(len(query_corpus)):

        # Gets the query name from the corresponding list
        name = query_names[query_indx]

        # Gets the query terms from the corresponding list
        query = query_corpus[query_indx]

        # Executes the query in the inverted index
        result = inverted_index_query(inverted_index, query, 331, True)

        # Writes the name into the file
        conj_file.write(name + '\t')

        # Writes relevant documents' ID into the file
        for r in result:
            conj_file.write('d' + str(r))

            # Condition to write or not a comma
            if r != result[-1]:
                conj_file.write(',')

        # Writes a new line
        conj_file.write('\n')
        
    # Calculates final time
    cumulative_time += time.time() - initial_time

    # Closes the file
    conj_file.close()
    
print("Average time: {} s".format(cumulative_time / N))

Average time: 0.0015923261642456054 s


In [12]:
""" Executes the disjuct queries and writes the results into a file.

Args:
    query_corpus (list): has lists whose elements are the terms used in the query.
    query_names (list): contains the names of the queries.
    
"""

# Sets a variable with the number of iterations to average
N = 10

# Initializes time value
cumulative_time = 0

# Runs the queries for a given time
for iteration in range(N):

    # Sets initial time
    initial_time = time.time()

    # Opens the results file
    conj_file = open("./results/BSII-OR-queries-results.tsv", "w")

    for query_indx in range(len(query_corpus)):
        

        # Gets the query name from the corresponding list
        name = query_names[query_indx]

        # Gets the query terms from the corresponding list
        query = query_corpus[query_indx]

        # Executes the query in the inverted index
        result = inverted_index_query(inverted_index, query, 331, False)

        # Writes the name into the file
        conj_file.write(name + '\t')

        # Writes relevant documents' ID into the file
        for r in result:
            conj_file.write('d' + str(r))

            # Condition to write or not a comma
            if r != result[-1]:
                conj_file.write(',')

        # Writes a new line
        conj_file.write('\n')
        
    # Calculates final time
    cumulative_time += time.time() - initial_time

    # Closes the file
    conj_file.close()

print("Average time: {} s".format(cumulative_time / N))

Average time: 1.7182111740112305 ms
